In [1]:
import numpy as np
from model import *
from keras.datasets import cifar10
from keras.utils import np_utils
from keras import metrics
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input, UpSampling2D, Dropout
from keras import metrics
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [2]:
dataset = np.load('x_train.npy')
labels = np.load('y_train.npy')

In [3]:
split_index = int(0.9 * len(dataset))
x_train, x_val = dataset[:split_index], dataset[split_index:]
y_train, y_val = labels[:split_index], labels[split_index:]

In [72]:
def create_low_param_model(input_shape, num_classes):
    cnn_in = layers.Input(shape=input_shape)
    cnn = layers.ZeroPadding1D(padding=4)(cnn_in)
    cnn = layers.Conv1D(filters=50, kernel_size=8, activation='relu')(cnn)
    cnn = layers.MaxPool1D(pool_size=2)(cnn)
    cnn = layers.Conv1D(filters=50, kernel_size=8, activation='relu')(cnn)
    cnn = layers.MaxPool1D(pool_size=2)(cnn)
    cnn = layers.Conv1D(filters=32, kernel_size=4, activation='relu')(cnn)
    cnn = layers.Dropout(rate=0.6)(cnn)
    cnn = layers.MaxPool1D(pool_size=2)(cnn)
    cnn = layers.Flatten()(cnn)
    cnn = layers.Dense(30, activation='selu')(cnn)
    cnn_out = layers.Dense(num_classes, activation='softmax')(cnn)
    
    model = Model(cnn_in, cnn_out)

    return model

In [73]:
model = create_low_param_model((1024, 2), 24)
opt = Adam(learning_rate=0.0007)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [76]:
num_epochs = 20

mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy',save_weights_only=True)
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, verbose=1, epsilon=1e-4, mode='min')

history = model.fit(x_train,
                    y_train,
                    epochs=num_epochs,
                    batch_size=128,
                    callbacks = [reduce_lr_loss,mcp_save],
                    validation_data=(x_val, y_val))

Epoch 1/20
2212/2212 [==============================] - 408s 185ms/step - loss: 1.2972 - accuracy: 0.4789 - val_loss: 1.0736 - val_accuracy: 0.5453
Epoch 2/20
2212/2212 [==============================] - 404s 183ms/step - loss: 0.9922 - accuracy: 0.5752 - val_loss: 1.0010 - val_accuracy: 0.5676
Epoch 3/20
2212/2212 [==============================] - 406s 184ms/step - loss: 0.8661 - accuracy: 0.6373 - val_loss: 0.8337 - val_accuracy: 0.6425
Epoch 4/20
2212/2212 [==============================] - 406s 183ms/step - loss: 0.7009 - accuracy: 0.7057 - val_loss: 0.6682 - val_accuracy: 0.7258
Epoch 5/20
2212/2212 [==============================] - 403s 182ms/step - loss: 0.5907 - accuracy: 0.7498 - val_loss: 0.5363 - val_accuracy: 0.7775
Epoch 6/20
2212/2212 [==============================] - 404s 183ms/step - loss: 0.5226 - accuracy: 0.7746 - val_loss: 0.5196 - val_accuracy: 0.7754
Epoch 7/20
2212/2212 [==============================] - 402s 182ms/step - loss: 0.4848 - accuracy: 0.7901 - val_

In [75]:
model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 1024, 2)]         0         
_________________________________________________________________
zero_padding1d_12 (ZeroPaddi (None, 1032, 2)           0         
_________________________________________________________________
conv1d_52 (Conv1D)           (None, 1025, 50)          850       
_________________________________________________________________
max_pooling1d_52 (MaxPooling (None, 512, 50)           0         
_________________________________________________________________
conv1d_53 (Conv1D)           (None, 505, 50)           20050     
_________________________________________________________________
max_pooling1d_53 (MaxPooling (None, 252, 50)           0         
_________________________________________________________________
conv1d_54 (Conv1D)           (None, 249, 32)           6432

In [77]:
model.save('first_model.h5')

Yukarıdaki kod yardımıyla parametre sayısı 147.146 olan derin öğrenme modeli oluşturulmuştur. Oluşturulan modelin eğitim accuracy değeri %85.99, validasyon skoru ise %88.09'dur. Model eğitildikten sonra 'first_model.h5' ismiyle kaydedilmiştir. Daha sonra modelin yeniden oluşturulabilmesi ve test için gerekli data işlemleri yapılabilmesi için 'model_load.ipynb' dosyası yazılmıştır.